## Install all dependencies

In [10]:
!pip install PyPDF2
!pip install langchain
!pip install pypdf 
!pip install ollama
!pip install pandas
!pip install openai
!pip install tiktoken
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install --force-reinstall ~/repos/pana/dist/pana-1.0-py3-none-any.whl

Defaulting to user installation because normal site-packages is not writeable
Processing /home/ztkpat001/repos/pana/dist/pana-1.0-py3-none-any.whl
  Attempting uninstall: pana
    Found existing installation: pana 1.0
    Uninstalling pana-1.0:
      Successfully uninstalled pana-1.0


In [12]:
import os
from dotenv import load_dotenv
# import necessary libraries
from pana import ModelFactory, PromptFlow
from pana.texts import nato_text_short, json_nato_flashcards, json_system_message
from langchain_community.document_loaders import PyPDFLoader


### Models

In [4]:
loader = PyPDFLoader("data/The_New_Invisible_Competitors.pdf")
pages = loader.load_and_split()
pages[0]
# Chunk the content into pieces 
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2000, chunk_overlap=100, add_start_index=True
)
all_splits = text_splitter.split_documents(pages)
print("Number of chunks:", len(all_splits))
print(all_splits[1].page_content)

Number of chunks: 16
02/02/2024, 19:48 The New Invisible Competitors | Wilson Quarterly
https://www.wilsonquarterly.com/quarterly/fall-2007-overdrive-competition-in-american-life/the-new-invisible-competitors 2/10Remember the Archie comics? Archie and his conceited rival Reggie battle for
the affections of Betty and Veronica, and the two girls, though they are best
friends, jockey for the attention of Archie, the affable  all- American boy. They
have been at it for more than 60 years, and in the early days the basic situation
wasn’t far removed from the experience of many Americans, especially in
small towns. Indeed, cartoonist Bob Montana based the Archie characters on
people he knew from his high school days during the 1930s in Haverhill,
Massachusetts. Most romantic competition occurred within small groups of
people who knew one another. The girl or guy would choose, perhaps the
couple would marry and settle down, and often the loser ended up living down
the street or across town. R

In [5]:
print(nato_text_short)
print(json_nato_flashcards)
print(json_system_message)


Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined.  


{"1":{"Question":"How many members are in the NATO Alliance?", "Answer":"30"}, "2":{"Question":"What is the name of the Ukrainian President?", "Answer":"Volodymyr Zelensky"}, "3":{"Question":"Who did the speak

In [6]:
json_flow = PromptFlow("json_flow", "Original flow with JSON output now")
json_flow.add_system_message(json_system_message)
print(json_flow)
json_flow.add_interaction("user",
                          f"""
                          {nato_text_short}
                          
                          Make 5 flashcards from the text above.
                          """
                          )
json_flow.add_interaction("assistant", json_nato_flashcards)
print(json_flow)
json_flow.save_to_file()

[
    {
        "role": "system",
        "content": "\nYou are a helpful study assistant. I want you to create flash cards to be used for studying. The cards should obey these axioms:\n\n        - Most flash cards should be atomic, i.e. they focus on a single piece of information.\n        - Make sure that that some of the flash cards are trivial so that users are able to build up key context on the topic.\n        - Avoid isolated flash cards which are not connected to other flash cards, that is ensure that the flash cards are connected to each other in such a way that when taken together they build up understanding.\n        - Do not, under any circumstances, make flash cards containing information which is not found in the provived text.\n        - The flash cards should be less than 8 words long.\n        \n        and have this JSON format:\n\n        \"{ \"X\":{\"Question\":Y, \"Answer\":Z}, ...}\"\n\n        Where X is the number of the flash card, Y is the question and Z is th

In [7]:
i = 2
json_flow_temp = json_flow
json_flow_temp.add_interaction("user", 
f"""
    Based on the following text:

    {all_splits[i].page_content}

    Make 2 flash cards.
"""
)
print(json_flow_temp)

[
    {
        "role": "system",
        "content": "\nYou are a helpful study assistant. I want you to create flash cards to be used for studying. The cards should obey these axioms:\n\n        - Most flash cards should be atomic, i.e. they focus on a single piece of information.\n        - Make sure that that some of the flash cards are trivial so that users are able to build up key context on the topic.\n        - Avoid isolated flash cards which are not connected to other flash cards, that is ensure that the flash cards are connected to each other in such a way that when taken together they build up understanding.\n        - Do not, under any circumstances, make flash cards containing information which is not found in the provived text.\n        - The flash cards should be less than 8 words long.\n        \n        and have this JSON format:\n\n        \"{ \"X\":{\"Question\":Y, \"Answer\":Z}, ...}\"\n\n        Where X is the number of the flash card, Y is the question and Z is th

In [14]:
load_dotenv()
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
model = ModelFactory.get_model("gpt-3.5-turbo-0125", api_key=OPENAI_API_KEY)

In [15]:
response = model.make_call(json_flow_temp.flow)

Established connection with gpt-3.5-turbo-0125
Calling gpt-3.5-turbo-0125 with prompt: [{'role': 'system', 'content': '\nYou are a helpful study assistant. I want you to create flash cards to be used for studying. The cards should obey these axioms:\n\n        - Most flash cards should be atomic, i.e. they focus on a single piece of information.\n        - Make sure that that some of the flash cards are trivial so that users are able to build up key context on the topic.\n        - Avoid isolated flash cards which are not connected to other flash cards, that is ensure that the flash cards are connected to each other in such a way that when taken together they build up understanding.\n        - Do not, under any circumstances, make flash cards containing information which is not found in the provived text.\n        - The flash cards should be less than 8 words long.\n        \n        and have this JSON format:\n\n        "{ "X":{"Question":Y, "Answer":Z}, ...}"\n\n        Where X is th

In [16]:
print(response)

ChatCompletion(id='chatcmpl-8scEAQ1zJg5K60XL7g0coPs8yte8J', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"1":{"Question":"From which countries are eager software programmers now coming?", "Answer":"India and Bangladesh"}, "2":{"Question":"What website allows easy hiring of specialists overseas?", "Answer":"Guru.com"}}', role='assistant', function_call=None, tool_calls=None))], created=1708027350, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_69829325d0', usage=CompletionUsage(completion_tokens=43, prompt_tokens=707, total_tokens=750))


In [ ]:
### NOW CONVERT INTO JSON